# Contributed by Stephen Spradling, Anna Koduru, & Logan Olbrich

In [1]:
%%time
%%capture
!pip install vaderSentiment
!pip install altair vega_datasets

CPU times: user 130 ms, sys: 39 ms, total: 169 ms
Wall time: 15.7 s


In [2]:
import pandas as pd
import numpy as np
import altair as alt
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json

In [3]:
base_url = "http://underdog-devs-ds-a-dev.us-east-1.elasticbeanstalk.com"

In [4]:
feedback_df = pd.DataFrame(requests.post(f"{base_url}/Feedback/read").json()["result"])[['ticket_id', 'mentor_id','mentee_id', 'feedback']]
mentee_df = pd.DataFrame(requests.post(f"{base_url}/Mentees/read").json()["result"])[['profile_id', 'first_name', 'last_name']]
mentor_df = pd.DataFrame(requests.post(f"{base_url}/Mentors/read").json()["result"])[['profile_id', 'first_name', 'last_name']]

In [12]:
mentee_df.rename(columns={'profile_id': 'mentee_id', 'first_name': 'mentee_first_name', 'last_name': 'mentee_last_name'}, inplace=True)
mentor_df.rename(columns={'profile_id': 'mentor_id', 'first_name': 'mentor_first_name', 'last_name': 'mentor_last_name'}, inplace=True)


In [13]:
mentor_df['mentor_full_name'] = mentor_df.mentor_first_name.str.cat(mentor_df.mentor_last_name, sep=" ")
mentee_df['mentee_full_name'] = mentee_df.mentee_first_name.str.cat(mentee_df.mentee_last_name, sep=" ")

In [14]:
df_1 = pd.merge(feedback_df, mentee_df, on='mentee_id', how='left')
mentor_feedback_df = pd.merge(df_1, mentor_df, on='mentor_id', how='left')

In [15]:
mentor_feedback_df['datetime'] = np.random.choice(pd.date_range('2020-01-01', '2021-01-01'), len(mentor_feedback_df))

In [16]:
analyzer = SentimentIntensityAnalyzer()

In [17]:
def feedback_outcome(text: str):
    '''
    Return an intuitive string
    (Positive, Negative, Neutral)
    base on the compound score of text from
    vader analysis.
    '''
    
    score = analyzer.polarity_scores(text)['compound']
    if score > 0.3:
        return 'Positive'
    elif score < -0.3:
        return 'Negative'
    else:
        return 'Neutral'

In [18]:
mentor_feedback_df['feedback_outcome'] = mentor_feedback_df['feedback'].apply(feedback_outcome)

In [19]:
mentor_feedback_df['vader_score'] = mentor_feedback_df['feedback'].apply(lambda x:analyzer.polarity_scores(x)['compound'])

In [20]:
mentor_feedback_df.head()

,ticket_id,mentor_id,mentee_id,feedback,mentee_first_name,mentee_last_name,mentee_full_name,mentor_first_name,mentor_last_name,mentor_full_name,datetime,feedback_outcome,vader_score
0,RGY31mG14606sC3B,UY340CfO5dJ2S210,8v417YeUI850jS3Z,I have many hours of undergraduate accounting ...,Brady,Parker,Brady Parker,Jerry,Reed,Jerry Reed,2020-02-01,Positive,0.7351
1,1R4587z1caGB1B7u,630lhQ431nA0Nz3s,yIBKz628y1Ol4538,Great introduction to Accounting Analytics. Un...,Bo,Reed,Bo Reed,Layne,Morris,Layne Morris,2020-09-27,Positive,0.3400
2,21x781Q7Ayo4mgE8,8lC6bv1xtG1046M6,4A763H2DOZxo68Z3,Very useful and comprehensive,Matthew,Miller,Matthew Miller,Sabrina,Richardson,Sabrina Richardson,2020-06-02,Positive,0.6659
3,20gp7c24Fx6h4D6g,J4o388P4plF5RY70,q742317EX47IcugF,This is a really good survey class in the pote...,Caden,Miller,Caden Miller,Salvatore,Brooks,Salvatore Brooks,2020-11-09,Positive,0.8250
4,46P6Sw8gLI5Q2N14,6602Q4ljE112heWg,b77WF13NE25b3h8F,"Nice course, a difficult one but very challeng...",Sarai,Price,Sarai Price,Johnny,Campbell,Johnny Campbell,2020-02-29,Positive,0.9466


In [21]:
source = mentor_feedback_df

In [22]:
bar = alt.Chart(source, title="Global Feedback & Average of Feedback Positivity"
    ).mark_bar(
    ).encode(
    x='datetime:T',
    y='vader_score:Q',
    color=alt.condition(
         alt.datum.vader_score > 0,
         alt.value("blue"),  # The positive color
         alt.value("orange")  # The negative color
    )
  )
line = alt.Chart(source).mark_line(color='red').transform_window(
    # The field to average
    rolling_mean='mean(vader_score)',
    # The number of values before and after the current value to include.
    # [null, 0] is the default value for frame.
    # frame=[-9, 0]
).encode(
    x='datetime',
    y='rolling_mean:Q'
)

(bar + line).properties(
    width=900).configure_title(fontSize=26).configure(background='#D9E9F0'
    ).interactive()

alt.LayerChart(...)

# Using this graph for overall feedback

In [64]:
interval = alt.selection_interval(encodings=['x'])
selection = alt.selection_multi(fields=['mentor_full_name'], bind='legend')

bar = alt.Chart(source, title="Global Feedback & Average of Feedback Positivity"
                ).mark_bar().encode(
    alt.X('datetime', title='Time'),
    alt.Y('vader_score', title='Positivity of Feedback by Mentees'),
    color=alt.condition(interval, alt.Color('mentor_full_name:N',
                                            title="Mentor",
                                            legend=alt.Legend(orient="left")),
                        alt.value('lightgray'))
).properties(
    width=900,
    selection=interval
).add_selection(
    selection
)
mean_line = alt.Chart(source).mark_line(color='red').transform_window(
    # The field to average
    mentor_progress='mean(vader_score)',
    # The number of values before and after the current value to include.
    frame=[-9, 0]
).encode(
    alt.X('datetime', title='Time'),
    alt.Y("mean(vader_score)", title="Progress of Mentors")
).properties(
    width=900
).transform_filter(
     interval
)

(bar & mean_line).configure_title(fontSize=20
                                             ).configure(background='#D9E9F0')

alt.VConcatChart(...)

# We have decided to use this graph for individual feedback of Mentors

In [62]:
selection = alt.selection_multi(fields=['mentor_full_name'], on='mouseover', bind='legend')


mentor_bar = alt.Chart(source, title="Individual Feedback Positivity Scores About a Mentor").mark_bar().encode(
    alt.X('datetime', title='Time'),
    alt.Y('vader_score', title='Positivity of Feedback by Mentees'),
    color=alt.condition(selection, alt.Color('mentor_full_name:N',
                                             title="Mentor",
                                             legend=alt.Legend(orient="left")),
                        alt.value('lightgray')),
    
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=[alt.Tooltip('mentor_full_name', title='Mentor'),
             alt.Tooltip('feedback_outcome', title='Feedback Outcome'),
             alt.Tooltip('mentee_full_name', title='Mentee'),
             alt.Tooltip('feedback', title='Feedback Given')]
).configure_range(
    category={'scheme': 'dark2'}
).properties(
    width=900,
    selection=selection
).add_selection(
    selection
)

mentor_bar.configure_title(fontSize=26).configure(background='#D9E9F0'
           ).interactive()

alt.Chart(...)